In [1]:
import pandas as pd
import numpy as np
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.support.wait import WebDriverWait
import time
options = webdriver.ChromeOptions()  
options.add_argument("start-maximized") # untuk layar chrome yang dibuka maximize. (search bar tidak keluar jika tidak maximize)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [2]:
driver = webdriver.Chrome(options=options, executable_path=r'C:/Program Files (x86)/chromedriver (2).exe')

In [3]:
driver.get('https://www.linkedin.com/')

#### Login

In [4]:
username = WebDriverWait(driver,10).until(EC.element_to_be_clickable((By.CSS_SELECTOR,"input[name='session_key']")))
password = WebDriverWait(driver,10).until(EC.element_to_be_clickable((By.CSS_SELECTOR,"input[name='session_password']")))
username.clear()
password.clear()
email = (input('Enter your email to login:'))
psw = (input('Enter your password to login:'))
username.send_keys(email)
password.send_keys(psw)

Enter your email to login: johanklemantan@gmail.com
Enter your password to login: funicula8


In [5]:
log_in = WebDriverWait(driver,10).until(EC.element_to_be_clickable((By.CSS_SELECTOR,"button[type='submit']"))).click()

#### Harus running 4x dan di menghasilkan 4 csv file

keyword list for this project :
- Senior Data Engineer / Data Engineer
- Senior Data Scientist / Data Scientist
- Senior Data Analyst / Data Analyst
- Senior Business Intelligence / Business Intelligence Analyst

#### Membuat link sesuai dengan keyword

In [6]:
keywords = (input('masukkan keyword:'))
keyword = [keywords]
key_save = keywords.replace('/','or')
def convert(keyword):
    return ([i for item in keyword for i in item.split()])
result = convert(keyword)
cari = ''
for i in result:
    cari+=(i)+'%20'
cari=cari[:-3]
cari

masukkan keyword: Senior Business Intelligence / Business Intelligence Analyst


'Senior%20Business%20Intelligence%20/%20Business%20Intelligence%20Analyst'

In [7]:
driver.get(f'https://www.linkedin.com/jobs/search/?geoId=102478259&keywords={cari}&location=Indonesia')

#### Scraping Data yang dibutuhkan :
1. Job Company
2. Job Posting Time
3. Career Level
4. Size of Company
5. Company Industry
6. Detail Description
7. Employment Type
8. Job Function

In [8]:
hasil = []
link = driver.current_url
jumlah_loker = int(driver.find_element_by_class_name('.'.join("display-flex t-12 t-black--light t-normal".split())).text.split(' ')[0])
print(f'Untuk posisi {keywords} ada {jumlah_loker} lowongan.')
# Jika halaman lebih dari 1(loker lebih dari 25), maka akan di looping sejumlah (25*jumlah halaman terakhir-1) + jumlah vacancy di halaman terakhir
if jumlah_loker > 25:
    while True:
        try:
            npages = int(driver.find_element_by_class_name("artdeco-pagination__pages.artdeco-pagination__pages--number").find_elements_by_tag_name("li")[-1].text)
            break
        except:
            pass

    for i in range(0, npages*25, 25):
        print("Page:",int((i/25)+1),"of",npages)
        #mengedit link buat ganti ke halaman berikutnya (ditambahin '&start=25 atau 50 atau 75' tergantung jumlah loker)
        driver.get(link+"&start="+str(i))
        time.sleep(1)
        loop = 0
        while True: 
            try: #mengklik lowongan agar keluar description yang lebih lengkap
                card_list = driver.find_elements_by_class_name('.'.join("disabled ember-view job-card-container__link job-card-list__title".split()))
                current_card = card_list[loop]
                loop+=1
                current_card.click() 
                time.sleep(3)
                try: #mulai scraping company name dan posted date (1 & 2)
                    print(f'page{int((i/25)+1)}, loop {loop}')
                    top_card = driver.find_element_by_class_name("jobs-details-top-card__content-container").text.split('\n')
                    for i in range(len(top_card)):
                        if "Company Name" == top_card[i]:
                            companyName = top_card[i+1]
                        elif "Posted Date" == top_card[i]:
                            postedDate = top_card[i+1]
                            break
                            #scraping company size (4)
                    companyInfo = driver.find_element_by_class_name("artdeco-list__item.jobs-details-job-summary__section.jobs-details-job-summary__section--center").text
                    sizeOfcompany = np.nan if "employees" not in companyInfo else companyInfo.split('\n')[1]
                    jobDescDetail = driver.find_element_by_class_name("jobs-description-details.pt4").find_elements_by_tag_name("div")
                    dataHead = ["seniority level", "industry", "employment type", "job functions"]
                    dataHeadInDetail = [x.text.lower().split('\n')[0] for x in jobDescDetail]
                    for desc in jobDescDetail:
                        head = desc.text.lower().split('\n')[0]
                        print(head)
                        if len(desc.find_elements_by_tag_name("li")) == 0:
                            value = desc.text.split('\n')[1]
                        else: #memasukkan value ke dalam variabel yang diinginkan (careerlevel, employment type, jobfunction, industry) supaya bisa dimasukkan ke dalam dataframe (3,5,7,8)
                            value = []
                            for x in desc.find_elements_by_tag_name("li"):
                                value.append(x.text)
                            value = value[0] if len(value) == 1 else value
                        if head == dataHeadInDetail[0]:
                            seniorityLevel = value
                            print(f'value {value}')
                        elif head == dataHeadInDetail[1]:
                            employmentType = value
                            print(f'value {value}')
                        elif head == dataHeadInDetail[2]:
                            jobFunction = value
                            print(f'value {value}')
                        elif head == dataHeadInDetail[3]:
                            companyIndustry = value 
                            print(f'value {value}')
                    headNotFound = list(set(dataHead)-set(dataHeadInDetail))
                    if len(headNotFound) > 0:
                        for head in headNotFound:
                            if head == dataHead[0]:
                                seniorityLevel = np.nan
                            elif head == dataHead[1]:
                                employmentType = np.nan 
                            elif head == dataHead[2]:
                                jobFunction = np.nan 
                            elif head == dataHead[3]:
                                companyIndustry = np.nan
                    #scraping jobdescription (6)
                    #beberapa jobdescription keluar '2nd' karena sama - sama memiliki tag 'span' di dalam id 'job-details' dan berada lebih atas dari tag 'span' yang berisi job description. 
                    detailDescription = driver.find_element_by_id("job-details").find_element_by_tag_name("span").text
                    hasil.append([companyName,postedDate,seniorityLevel,sizeOfcompany,companyIndustry,detailDescription,employmentType,jobFunction])
                except:
                    pass
            except:
                break
#jika jumlah loker kurang dari 25 (halamannya cuman 1. gapake looping ke halaman berikutnya)
else:
    loop = 0
    while True: 
        try:
            card_list = driver.find_elements_by_class_name('.'.join("disabled ember-view job-card-container__link job-card-list__title".split()))
            current_card = card_list[loop]
            loop+=1
            current_card.click() 
            time.sleep(3)
            try:
                print(f'loop {loop}')
                top_card = driver.find_element_by_class_name("jobs-details-top-card__content-container").text.split('\n')
                for i in range(len(top_card)):
                    if "Company Name" == top_card[i]:
                        companyName = top_card[i+1]
                    elif "Posted Date" == top_card[i]:
                        postedDate = top_card[i+1]
                        break
                companyInfo = driver.find_element_by_class_name("artdeco-list__item.jobs-details-job-summary__section.jobs-details-job-summary__section--center").text
                sizeOfcompany = np.nan if "employees" not in companyInfo else companyInfo.split('\n')[1]
                jobDescDetail = driver.find_element_by_class_name("jobs-description-details.pt4").find_elements_by_tag_name("div")
                dataHead = ["seniority level", "industry", "employment type", "job functions"]
                dataHeadInDetail = [x.text.lower().split('\n')[0] for x in jobDescDetail]
                for desc in jobDescDetail:
                    head = desc.text.lower().split('\n')[0]
                    print(head)
                    if len(desc.find_elements_by_tag_name("li")) == 0:
                        value = desc.text.split('\n')[1]
                    else:
                        value = []
                        for x in desc.find_elements_by_tag_name("li"):
                            value.append(x.text)
                        value = value[0] if len(value) == 1 else value
                    if head == dataHeadInDetail[0]:
                        seniorityLevel = value
                        print(f'value {value}')
                    elif head == dataHeadInDetail[1]:
                        employmentType = value
                        print(f'value {value}')
                    elif head == dataHeadInDetail[2]:
                        jobFunction = value
                        print(f'value {value}')
                    elif head == dataHeadInDetail[3]:
                        companyIndustry = value 
                        print(f'value {value}')
                headNotFound = list(set(dataHead)-set(dataHeadInDetail))
                if len(headNotFound) > 0:
                    for head in headNotFound:
                        if head == dataHead[0]:
                            seniorityLevel = np.nan
                        elif head == dataHead[1]:
                            employmentType = np.nan 
                        elif head == dataHead[2]:
                            jobFunction = np.nan 
                        elif head == dataHead[3]:
                            companyIndustry = np.nan
                detailDescription = driver.find_element_by_id("job-details").find_element_by_tag_name("span").text
                hasil.append([companyName,postedDate,seniorityLevel,sizeOfcompany,companyIndustry,detailDescription,employmentType,jobFunction])
            except:
                pass
        except:
            break

Untuk posisi Senior Business Intelligence / Business Intelligence Analyst ada 17 lowongan.
loop 1
loop 2
seniority level
value Associate
industry
value ['Computer Software', 'Internet', 'Financial Services']
employment type
value Full-time
job functions
value ['Research', 'Analyst', 'Information Technology']
loop 3
seniority level
value Mid-Senior level
industry
value Internet
employment type
value Full-time
job functions
value ['Finance', 'Strategy/Planning']
loop 4
seniority level
value Associate
industry
value ['Information Technology & Services', 'Computer Software', 'Retail']
employment type
value Full-time
job functions
value ['Business Development', 'Sales']
loop 5
seniority level
value Associate
industry
value ['Restaurants', 'Food & Beverages', 'Food Production']
employment type
value Full-time
job functions
value ['Research', 'Analyst', 'Information Technology']
loop 6
seniority level
value Mid-Senior level
industry
value ['Food & Beverages', 'Logistics & Supply Chain', 'Reta

In [9]:
df = pd.DataFrame(hasil, columns = ['companyName','postedDate','careerLevel','sizeOfEmployee','companyIndustry','detailDescription','employmentType','jobFunction'])
df

,companyName,postedDate,careerLevel,sizeOfEmployee,companyIndustry,detailDescription,employmentType,jobFunction
0,Gojek,Posted 1 week ago,Associate,"1,001-5,000 employees","[Research, Analyst, Information Technology]",About The Role\n\nWe'll cut to the chase and g...,"[Computer Software, Internet, Financial Services]",Full-time
1,Tokopedia,Posted 2 weeks ago,Mid-Senior level,"1,001-5,000 employees","[Finance, Strategy/Planning]",Job Description:\nDevelop and continually impr...,Internet,Full-time
2,Soyaka AI (Shox),Posted 3 weeks ago,Associate,NaN,"[Business Development, Sales]",Requirements\nBachelor's Degree in Computer Sc...,"[Information Technology & Services, Computer S...",Full-time
3,HAVI,Posted 2 months ago,Associate,10001+ employees,"[Research, Analyst, Information Technology]",Overview\n\nWe support applications that cover...,"[Restaurants, Food & Beverages, Food Production]",Full-time
4,Kopi Kenangan,Posted 3 days ago,Mid-Senior level,"1,001-5,000 employees","[Information Technology, Engineering, Analyst]",2nd,"[Food & Beverages, Logistics & Supply Chain, R...",Full-time
5,Agoda,Posted 2 weeks ago,Associate,"1,001-5,000 employees",Other,About Agoda\n\nAgoda is an online travel booki...,Internet,Full-time
6,"Love, Bonito Indonesia",Posted 1 week ago,Associate,NaN,Information Technology,You’ll have a front seat experience in impacti...,Retail,Full-time
7,HAVI,Posted 3 months ago,NaN,10001+ employees,Information Technology,Overview\n\nWe support applications that cover...,Full-time,"[Research, Analyst, Information Technology]"
8,JULO,Posted 2 weeks ago,Associate,NaN,"[Research, Analyst, Information Technology]",Job Description\n\n& requirements\nWe are look...,"[Information Technology & Services, Computer S...",Full-time
9,Havi,Posted 1 week ago,Associate,NaN,"[Research, Analyst, Information Technology]",Overview\n\nWe support applications that cover...,Warehousing,Full-time


In [10]:
df.to_csv(f'{key_save}.csv',index=False)